In [1]:
import calendar
import pandas as pd
import panel as pn
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import show
pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 2, 19)

### Tables in the process

In [2]:
cols = 'trade name qty price active reason market'.split()
colt = 'trans name spd reason qty target current percent active'.split()

In [3]:
format_dict = {
    'qty':'{:,}','available_qty':'{:,}',
    'price':'{:.2f}','cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}','target':'{:.2f}','current':'{:.2f}',
    'beta':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}','volume':'{:,.2f}',
    'percent':'{:,.2f}'
}

In [4]:
sql = '''
SELECT *
FROM orders
ORDER BY id DESC
LIMIT 1'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

,id,trade,name,qty,price,active,reason,market
0,40,S,WHART,"10,000",12.80,1,DOS,SET


In [5]:
names = orders['name']
name = names.to_string(index=False)

sql = """
SELECT * FROM stocks WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * FROM stocks WHERE name = 'WHART'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,687,WHART,14.08,10.68,I,0.00,12.80,20.36,0.39,12.50,"10,000",-5,4,"10,000",0,0,DOS,SET


### Create orders from stocks after adjust stocks in port_lite from consensus

In [6]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

41

In [7]:
sql = '''
INSERT INTO orders
SELECT id, 'B', name, qty, buy_target, 0, reason, market
FROM stocks
WHERE status IN ('B','O')
ORDER BY name'''
rp = conlite.execute(sql)
rp.rowcount

16

In [8]:
sql = '''
INSERT INTO orders
SELECT id,'S',name, qty, sell_target, 0, reason, market
FROM stocks
WHERE status IN ('I','S')'''
rp = conlite.execute(sql)
rp.rowcount

20

In [9]:
sql = '''
SELECT *
FROM orders
ORDER BY trade, name'''
df_orders = pd.read_sql(sql, conlite)
df_orders.shape[0]

36

In [10]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### After call Ord-log.bat

In [11]:
def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'

In [12]:
df = pd.read_csv('../data/orders-log.csv')
#df.rename(columns = {'col1': 'name', 'col2': 'spd','col3': 'reason', 'col4': 'market',
#                     'col5': 'qty','col6': 'target','col7': 'current', 'col8': 'chg',
#                     'col9': 'percent', 'col10': 'active'}, inplace = True)
df['trans'] = df.apply(lambda row: categorise(row), axis=1)

df_tab = pn.widgets.Tabulator(df, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trade    name  ..., width=800)

In [13]:
df[colt]

,trans,name,spd,reason,qty,target,current,percent,active
0,Buy,ASK,-10,1L,3000,42.00,44.50,-0.56%,0
1,Buy,ASP,-5,6L,30000,3.72,3.82,0.00%,0
2,Buy,BGC,-4,DOS,10000,10.00,10.40,-0.95%,0
3,Buy,CPNCG,-1,DOS,10000,12.40,12.50,+0.81%,0
4,Buy,KKP,-8,3L,3000,68.00,70.00,-1.41%,0
5,Buy,MAKRO,-3,3L,3000,42.00,42.75,+2.40%,0
6,Buy,MEGA,-12,RD05pct,3000,40.50,43.50,0.00%,0
7,Buy,NOBLE,-8,RD15%,21000,5.85,6.25,0.00%,0
8,Buy,PTTGC,-3,RD15pct,1200,54.25,55.00,-2.22%,0
9,Buy,ROJNA,-11,6L,18000,6.30,6.85,-2.14%,0
